# Global Experiment Setup

In [9]:
import mlflow


# Set the MLflow tracking URI to a new SQLite URI
mlflow.set_tracking_uri("sqlite:///new_mlflow.db")
mlflow.set_experiment("XGBoost")

<Experiment: artifact_location='/Users/arham/Downloads/Projects/03-Experiments/mlruns/2', creation_time=1713912394972, experiment_id='2', last_update_time=1713912394972, lifecycle_stage='active', name='XGBoost', tags={}>

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
import lightgbm as lgb
from sklearn.metrics import accuracy_score
import warnings
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score


def load_data(path):
    df = pd.read_csv(path)
    train_df, test_df = train_test_split(df, test_size=0.35, random_state=42)
    train_df, val_df,  = train_test_split(train_df, test_size=0.20, random_state=42)
    train_df = train_df.drop(['id'], axis=1).drop_duplicates().reset_index(drop=True)
    test_df = test_df.drop(['id'], axis=1).drop_duplicates().reset_index(drop=True)
    val_df = val_df.drop(['id'], axis=1).drop_duplicates().reset_index(drop=True)
    return train_df, val_df, test_df

def encode_target(train):
    target_key = {'Insufficient_Weight': 0, 'Normal_Weight': 1, 'Overweight_Level_I': 2, 'Overweight_Level_II': 3, 'Obesity_Type_I': 4,'Obesity_Type_II' : 5, 'Obesity_Type_III': 6}
    train['NObeyesdad'] = train['NObeyesdad'].map(target_key)
    return train

def make_gender_binary(train):
    train['Gender'] = train['Gender'].map({'Male':0, 'Female':1})

def datatypes(train):
    train['Weight'] = train['Weight'].astype(float)
    train['Age'] = train['Age'].astype(float)
    train['Height'] = train['Height'].astype(float)
    return train

# def age_binning(train_df):
#     # train_df['Age_Group'] = pd.cut(train_df['Age'], bins=[0, 20, 30, 40, 50, train_df['Age'].max()], labels=['0-20', '21-30', '31-40', '41-50', '50+'])
#     train_df['Age_Group'] = pd.cut(train_df['Age'], bins=[0, 20, 30, 40, 50, train_df['Age'].max()], labels=[1, 2, 3, 4, 5])
#     train_df['Age_Group'] = train_df['Age_Group'].astype(int)
#     return train_df

def age_binning(df):
    age_groups = []
    for age in df['Age']:
        if age <= 20:
            age_group = 1
        elif age <= 30:
            age_group = 2
        elif age <= 40:
            age_group = 3
        elif age <= 50:
            age_group = 4
        else:
            age_group = 5
        age_groups.append(age_group)
    df['Age_Group'] = age_groups
    return df

def age_scaling_log(train_df):
    train_df['Age'] = train_df['Age'].astype(float)
    train_df['Log_Age'] = np.log1p(train_df['Age'])
    return train_df

def age_scaling_minmax(train_df):
    train_df['Age'] = train_df['Age'].astype(float)
    scaler_age = MinMaxScaler()
    train_df['Scaled_Age'] = scaler_age.fit_transform(train_df['Age'].values.reshape(-1, 1))
    return train_df, scaler_age

def weight_scaling_log(train_df):
    train_df['Weight'] = train_df['Weight'].astype(float)
    train_df['Log_Weight'] = np.log1p(train_df['Weight'])
    return train_df

def weight_scaling_minmax(train_df):
    train_df['Weight'] = train_df['Weight'].astype(float)
    scaler_weight = MinMaxScaler()
    train_df['Scaled_Weight'] = scaler_weight.fit_transform(train_df['Weight'].values.reshape(-1, 1))
    return train_df, scaler_weight

def height_scaling_log(train_df):
    train_df['Log_Height'] = np.log1p(train_df['Height'])
    return train_df

def height_scaling_minmax(train_df):
    scaler_height = MinMaxScaler()
    train_df['Scaled_Height'] = scaler_height.fit_transform(train_df['Height'].values.reshape(-1, 1))
    return train_df, scaler_height

def make_gender_binary(train):
    train['Gender'] = train['Gender'].map({'Female':1, 'Male':0})
    return train

def fix_binary_columns(train):
    Binary_Cols = ['family_history_with_overweight','FAVC', 'SCC','SMOKE']
    # if yes then 1 else 0
    for col in Binary_Cols:
        train[col] = train[col].map({'yes': 1, 'no': 0})
        # column datatype integer
        train[col] = train[col].astype(int)
    return train

def freq_cat_cols(train):
    # One hot encoding
    cat_cols = ['CAEC', 'CALC']
    for col in cat_cols:
        train[col] = train[col].map({'no': 0, 'Sometimes': 1, 'Frequently': 2, 'Always': 3})
    return train

def Mtrans(train):
    """
    Public_Transportation    8692
    Automobile               1835
    Walking                   231
    Motorbike                  19
    Bike                       16
    """
    # train['MTRANS'] = train['MTRANS'].map({'Public_Transportation': 3, 'Automobile': 5, 'Walking': 1, 'Motorbike': 4, 'Bike': 2})
    # dummify column
    train = pd.get_dummies(train, columns=['MTRANS'])
    # convert these columns to integer
    train['MTRANS_Automobile'] = train['MTRANS_Automobile'].astype(int)
    train['MTRANS_Walking'] = train['MTRANS_Walking'].astype(int)
    train['MTRANS_Motorbike'] = train['MTRANS_Motorbike'].astype(int)
    train['MTRANS_Bike'] = train['MTRANS_Bike'].astype(int)
    train['MTRANS_Public_Transportation'] = train['MTRANS_Public_Transportation'].astype(int)
    return train


def other_features(train):
    train['BMI'] = train['Weight'] / (train['Height'] ** 2)
    # train['Age'*'Gender'] = train['Age'] * train['Gender']
    polynomial_features = PolynomialFeatures(degree=2)
    X_poly = polynomial_features.fit_transform(train[['Age', 'BMI']])
    poly_features_df = pd.DataFrame(X_poly, columns=['Age^2', 'Age^3', 'BMI^2', 'Age * BMI', 'Age * BMI^2', 'Age^2 * BMI^2'])
    train = pd.concat([train, poly_features_df], axis=1)
    return train


def test_pipeline(test, scaler_age, scaler_weight, scaler_height):
    test = datatypes(test)
    test = encode_target(test)
    test = age_binning(test)
    test = age_scaling_log(test)
    test['Scaled_Age'] = scaler_age.transform(test['Age'].values.reshape(-1, 1))
    test = weight_scaling_log(test)
    test['Scaled_Weight'] = scaler_weight.transform(test['Weight'].values.reshape(-1, 1))
    test = height_scaling_log(test)
    test['Scaled_Height'] = scaler_height.transform(test['Height'].values.reshape(-1, 1))
    test = make_gender_binary(test)
    test = fix_binary_columns(test)
    test = freq_cat_cols(test)
    test = Mtrans(test)
    test = other_features(test)

    return test

def train_model(params, X_train, y_train):
    lgb_train = lgb.Dataset(X_train, y_train)
    model = lgb.train(params, lgb_train, num_boost_round=1000)
    return model

def evaluate_model(model, X_val, y_val):
    y_pred = model.predict(X_val)
    y_pred = [np.argmax(y) for y in y_pred]
    accuracy = accuracy_score(y_val, y_pred)
    return accuracy

def objective(trial, X_train, y_train):
    params = {
        'objective': 'multiclass',
        'num_class': 7,
        'metric': 'multi_logloss',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.5),
        'num_leaves': trial.suggest_int('num_leaves', 10, 1000),
        'max_depth': trial.suggest_int('max_depth', -1, 20),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.6, 0.95),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.6, 0.95),
        'verbosity': -1
    }

    n_splits = 5
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    scores = []

    for train_index, val_index in kf.split(X_train, y_train):
        X_tr, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
        y_tr, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

        model = train_model(params, X_tr, y_tr)
        accuracy = evaluate_model(model, X_val, y_val)
        scores.append(accuracy)

    return np.mean(scores)

def optimize_hyperparameters(X_train, y_train, n_trials=2):
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=n_trials)
    return study.best_params


### XGB With Optuna

In [11]:

path = '/Users/arham/Downloads/Projects/01-Dataset/01-Data-for-model-building/train.csv'
train_df, val_df, test_df = load_data(path)

train_df = datatypes(train_df)
train_df = encode_target(train_df)
train_df = age_binning(train_df)
train_df, scaler_age = age_scaling_minmax(train_df)
train_df = age_scaling_log(train_df)
train_df, scaler_weight = weight_scaling_minmax(train_df)
train_df = weight_scaling_log(train_df)
train_df, scaler_height = height_scaling_minmax(train_df)
train_df = height_scaling_log(train_df)
train_df = make_gender_binary(train_df)
train_df = fix_binary_columns(train_df)
train_df = freq_cat_cols(train_df)
train_df = Mtrans(train_df)
train_df = other_features(train_df)

val_df = test_pipeline(val_df, scaler_age, scaler_weight, scaler_height)
test_df = test_pipeline(test_df, scaler_age, scaler_weight, scaler_height)

Target = 'NObeyesdad'
# features = train_df.columns.drop(Target)
features = ['Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight',
       'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE',
       'CALC', 'Age_Group', 
       'MTRANS_Automobile', 'MTRANS_Bike', 'MTRANS_Motorbike',
       'MTRANS_Public_Transportation', 'MTRANS_Walking', 'BMI', 'Age^2',
       'Age^3', 'BMI^2', 'Age * BMI', 'Age * BMI^2', 'Age^2 * BMI^2'] 

       #'Scaled_Age', 'Log_Age', 'Scaled_Weight', 'Log_Weight', 'Scaled_Height', 'Log_Height',


X_train = train_df[features]
y_train = train_df[Target]
X_val = val_df[features]
y_val = val_df[Target]
X_test = test_df[features]
y_test = test_df[Target]

# save X_train, y_train, X_val, X_test, y_test


In [12]:
import optuna
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, precision_recall_fscore_support
import mlflow

mlflow.sklearn.autolog(disable=True)

def objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_discrete_uniform('subsample', 0.6, 1, 0.1),
        'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree', 0.6, 1, 0.1),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 100.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 100.0),
        'random_state': 42,
        'objective': 'multi:softmax',
        'num_class': 7,
        'eval_metric': 'mlogloss'
    }

    with mlflow.start_run(run_name="XGB_with_Feature_Engineering_optuna_" + str(trial.number), nested=True):
        # Log hyperparameters as a single dictionary
        mlflow.log_params(params)

        # Calculate target drift and log
        class_counts_train = [y_train[y_train == i].count() / y_train.count() for i in range(7)]
        class_counts_val = [y_val[y_val == i].count() / y_val.count() for i in range(7)]
        target_drift = [(train_count - val_count) for train_count, val_count in zip(class_counts_train, class_counts_val)]
        mlflow.log_params({'Target_Drift_' + str(i): freq for i, freq in enumerate(target_drift)})
        print(f"Target Drift For Each Class {target_drift}")

        # Train XGBoost model
        model = XGBClassifier(**params)
        cv_scores_xgb = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
        print("Cross-validation Scores (XGBoost):", cv_scores_xgb)
        print("Mean CV Accuracy (XGBoost):", cv_scores_xgb.mean())

        # Log CV metrics
        mlflow.log_metric('mean_cv_accuracy', cv_scores_xgb.mean())

        # Fit model on full training data
        model.fit(X_train, y_train)
        y_val_pred_xgb = model.predict(X_val)

        # Calculate evaluation metrics
        accuracy_xgb = accuracy_score(y_val, y_val_pred_xgb)
        precision_xgb = precision_score(y_val, y_val_pred_xgb, average='weighted')
        recall_xgb = recall_score(y_val, y_val_pred_xgb, average='weighted')
        f1_xgb = 2 * (precision_xgb * recall_xgb) / (precision_xgb + recall_xgb)
        print("\nAccuracy (XGBoost):", accuracy_xgb)
        print("Precision (XGBoost):", precision_xgb)
        print("Recall (XGBoost):", recall_xgb)
        print("F1 (XGBoost):", f1_xgb)

        # Log evaluation metrics
        mlflow.log_metric('accuracy', accuracy_xgb)
        mlflow.log_metric('precision', precision_xgb)
        mlflow.log_metric('recall', recall_xgb)
        mlflow.log_metric('f1', f1_xgb)

        # Calculate and log precision, recall for each class
        precision_per_class, recall_per_class, _, _ = precision_recall_fscore_support(y_val, y_val_pred_xgb, average=None)
        for i in range(len(recall_per_class)):
            print(f"Recall for class {i}: {recall_per_class[i]}")
            mlflow.log_metric(f'recall_class_{i}', recall_per_class[i])

        # Log XGBoost model
        mlflow.xgboost.log_model(model, 'model')

        # Set tags
        mlflow.set_tag('experiments', 'Arham A.')
        mlflow.set_tag('model_name', 'XGBoost')
        mlflow.set_tag('preprocessing', 'Yes')

    return cv_scores_xgb.mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=15)

print('Number of finished trials:', len(study.trials))
print('Best trial:')
trial = study.best_trial
print('  Value: {:.5f}'.format(trial.value))
print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))


[I 2024-04-25 14:42:39,110] A new study created in memory with name: no-name-8728ed43-e2e9-48b2-9517-da58bc7e532c


Target Drift For Each Class [0.004943133623686147, 0.011990707821925795, -0.0087675011457998, -0.001077949504617301, -0.017190035106736085, -0.00032756263090533144, 0.01042920694244659]
Cross-validation Scores (XGBoost): [0.90597499 0.9078277  0.90088004 0.89944393 0.9012975 ]
Mean CV Accuracy (XGBoost): 0.9030848301461092

Accuracy (XGBoost): 0.9029270100037051
Precision (XGBoost): 0.9030076129712707
Recall (XGBoost): 0.9029270100037051
F1 (XGBoost): 0.9029673096887412
Recall for class 0: 0.930379746835443
Recall for class 1: 0.9090909090909091
Recall for class 2: 0.7582089552238805
Recall for class 3: 0.8389057750759878
Recall for class 4: 0.8646080760095012
Recall for class 5: 0.9688995215311005
Recall for class 6: 0.9960474308300395


[I 2024-04-25 14:47:04,672] Trial 0 finished with value: 0.9030848301461092 and parameters: {'max_depth': 7, 'learning_rate': 0.018865210874328926, 'n_estimators': 890, 'min_child_weight': 1, 'subsample': 1.0, 'colsample_bytree': 0.7, 'gamma': 0.05166504735637329, 'reg_alpha': 1.950318399401875e-07, 'reg_lambda': 0.0016535137882194538}. Best is trial 0 with value: 0.9030848301461092.


Target Drift For Each Class [0.004943133623686147, 0.011990707821925795, -0.0087675011457998, -0.001077949504617301, -0.017190035106736085, -0.00032756263090533144, 0.01042920694244659]
Cross-validation Scores (XGBoost): [0.90458546 0.90875405 0.90273275 0.89990732 0.90176089]
Mean CV Accuracy (XGBoost): 0.9035480933961377

Accuracy (XGBoost): 0.9003334568358652
Precision (XGBoost): 0.9005180080824259
Recall (XGBoost): 0.9003334568358652
F1 (XGBoost): 0.9004257230027402
Recall for class 0: 0.930379746835443
Recall for class 1: 0.9010695187165776
Recall for class 2: 0.764179104477612
Recall for class 3: 0.8358662613981763
Recall for class 4: 0.8551068883610451
Recall for class 5: 0.9665071770334929
Recall for class 6: 0.9960474308300395


[I 2024-04-25 14:47:24,807] Trial 1 finished with value: 0.9035480933961377 and parameters: {'max_depth': 4, 'learning_rate': 0.34412782834363764, 'n_estimators': 122, 'min_child_weight': 8, 'subsample': 0.7, 'colsample_bytree': 0.8, 'gamma': 0.7150632151328797, 'reg_alpha': 1.563625221482243, 'reg_lambda': 54.76107004673495}. Best is trial 1 with value: 0.9035480933961377.


...

Target Drift For Each Class [0.004943133623686147, 0.011990707821925795, -0.0087675011457998, -0.001077949504617301, -0.017190035106736085, -0.00032756263090533144, 0.01042920694244659]
Cross-validation Scores (XGBoost): [0.89207967 0.89671144 0.87772117 0.87998146 0.88461538]
Mean CV Accuracy (XGBoost): 0.8862218246270434

Accuracy (XGBoost): 0.8921822897369396
Precision (XGBoost): 0.8923780671270238
Recall (XGBoost): 0.8921822897369396
F1 (XGBoost): 0.892280167692982
Recall for class 0: 0.9367088607594937
Recall for class 1: 0.8877005347593583
Recall for class 2: 0.746268656716418
Recall for class 3: 0.8267477203647416
Recall for class 4: 0.8408551068883611
Recall for class 5: 0.9569377990430622
Recall for class 6: 0.9960474308300395


[I 2024-04-25 14:51:59,160] Trial 4 finished with value: 0.8862218246270434 and parameters: {'max_depth': 8, 'learning_rate': 0.013161640722756815, 'n_estimators': 117, 'min_child_weight': 10, 'subsample': 1.0, 'colsample_bytree': 0.9, 'gamma': 1.4696119827570218e-07, 'reg_alpha': 3.72742407162119e-05, 'reg_lambda': 3.979840692102382e-06}. Best is trial 3 with value: 0.9047520970689327.


Target Drift For Each Class [0.004943133623686147, 0.011990707821925795, -0.0087675011457998, -0.001077949504617301, -0.017190035106736085, -0.00032756263090533144, 0.01042920694244659]


[W 2024-04-25 14:53:28,183] Trial 5 failed with parameters: {'max_depth': 9, 'learning_rate': 0.10310165900206837, 'n_estimators': 423, 'min_child_weight': 8, 'subsample': 0.8, 'colsample_bytree': 0.7, 'gamma': 4.8147716158359994e-08, 'reg_alpha': 0.07526608599999886, 'reg_lambda': 0.00019135435817934415} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/arham/anaconda3/envs/DataScience/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/2t/c7s0z0zs4698zw0k9pj4f2r80000gn/T/ipykernel_8377/3506214257.py", line 39, in objective
    cv_scores_xgb = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
  File "/Users/arham/anaconda3/envs/DataScience/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)
  File "/Users/arham/anaconda3/envs/DataScience/lib/python3.10/site-packages/

KeyboardInterrupt: 

In [13]:
!mlflow ui --backend-store-uri "sqlite:////Users/arham/Downloads/Projects/03-Experiments/new_mlflow.db"

[2024-04-25 14:53:39 -0400] [18583] [INFO] Starting gunicorn 21.2.0
[2024-04-25 14:53:39 -0400] [18583] [INFO] Listening at: http://127.0.0.1:5000 (18583)
[2024-04-25 14:53:39 -0400] [18583] [INFO] Using worker: sync
[2024-04-25 14:53:39 -0400] [18584] [INFO] Booting worker with pid: 18584
[2024-04-25 14:53:39 -0400] [18585] [INFO] Booting worker with pid: 18585
[2024-04-25 14:53:39 -0400] [18586] [INFO] Booting worker with pid: 18586
[2024-04-25 14:53:39 -0400] [18587] [INFO] Booting worker with pid: 18587
^C
[2024-04-25 14:54:37 -0400] [18583] [INFO] Handling signal: int
[2024-04-25 14:54:37 -0400] [18585] [INFO] Worker exiting (pid: 18585)
[2024-04-25 14:54:37 -0400] [18587] [INFO] Worker exiting (pid: 18587)
[2024-04-25 14:54:37 -0400] [18584] [INFO] Worker exiting (pid: 18584)
[2024-04-25 14:54:37 -0400] [18586] [INFO] Worker exiting (pid: 18586)
